#Import data from url and convert to HTML format to access data for the Pandas dataframe using Beautiful Soup

In [18]:
import pandas as pd
import urllib.request
import json
from bs4 import BeautifulSoup
url =" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req=urllib.request.urlopen(url)
data=req.read().decode()

In [19]:
soup = BeautifulSoup(data,'html.parser')
table = soup.find('table',class_='sortable')

#Panda dataframe to extract header information

In [20]:
th = table.find_all('th')
th
headings = [t.text.strip() for t in th]
neighbourhoods = pd.DataFrame(columns=headings)
neighbourhoods

,Postcode,Borough,Neighbourhood


Extract detail information for the headers

In [21]:
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    pc,b,n = [td.text.strip() for td in tds]
    if b!="Not assigned":
        neighbourhoods = neighbourhoods.append({'Postcode':pc,'Borough':b,'Neighbourhood':n}, ignore_index=True)
neighbourhoods.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Combine neighbourhoods per postal code with "," seperators

In [22]:
neighbourhoods['Neighbourhood']=neighbourhoods.groupby('Postcode')['Neighbourhood'].transform(lambda x: "%s" % ','.join(x)).values
neighbourhoods

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
...,...,...,...
205,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."
206,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."
207,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."
208,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."


Remove duplicates

In [23]:
neighbourhoods=neighbourhoods.drop_duplicates().reset_index(drop=True)
neighbourhoods.head()
neighbourhoods

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."


In [24]:
neighbourhoods.shape

(103, 3)

Import geospatial data from given url

In [25]:
geospatial_data=pd.read_csv("https://cocl.us/Geospatial_data")
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
#Merge the 2 dataframes using Postal code

In [26]:
new_data=pd.merge(neighbourhoods,geospatial_data,how= 'left',left_on='Postcode',right_on='Postal Code',validate="1:1")
new_data.drop(labels='Postal Code',axis=1,inplace=True)
new_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [27]:
new_data.shape

(103, 5)